In [1]:
import numpy as np
!pip install opencv-python
import cv2 #OPEN CV
from matplotlib import pyplot as plt #to show the images

In [10]:
def process_img(image): #basically tuning the parameters before getting the contours  
    image = cv2.fastNlMeansDenoisingColored(image,None,18,18,11,35) #denoising
    Gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # turn gray   
    Blur = cv2.GaussianBlur(Gray,(7,7),1)
    Canny = cv2.Canny(Blur,150,150) 
    kernel = np.ones((3,3) , np.uint8)
    imgDial = cv2.dilate(Canny, kernel,iterations=5)
    return imgDial

In [11]:
def getContours(img):

    img_processed = process_img(img)
    contours, hierarchy = cv2.findContours(img_processed,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    # Create a mask for the convex hull
    mask = np.zeros_like(img)  
    result = img.copy() #we will work on result as to not alter original img
    
    #detect contour
    for i in contours: #look through contours
        area = cv2.contourArea(i) # get Area
        
        if area>800: #area threshold
            cv2.drawContours(result,[i],-1,(0,255,0),2) #draw green contour lines
            perimeter = cv2.arcLength(i, True)
            corner_pts_approx = cv2.approxPolyDP(i, 0.02*perimeter, True)
            objCor = len(corner_pts_approx)
            hull = cv2.convexHull(corner_pts_approx) #find the convex hull
            
            #fill the area below the convex hull
            cv2.fillConvexPoly(mask, hull, (255, 0, 0))
            
            weighted_clr =0.7 #value between 0 and 1 (can be tuned)
            
            #apply the mask
            filled = cv2.bitwise_and(result, mask)
            result = cv2.addWeighted(result, 1, filled, weighted_clr, 0)
            
            #the bounding box of the convex hull
            x, y, w, h = cv2.boundingRect(hull)
            
            filled_area = np.zeros_like(img)
            filled_area[y+h:, x:x+w] = (255, 0, 0)  #fill the box are below contour
            result = cv2.addWeighted(result, 1, filled_area, 0.3, 0)
            
    return result


In [12]:
f1 = cv2.imread(r"C:\Users\w10\Desktop\floor2.jpg")

In [13]:
c= getContours(f1)

In [6]:
#cv2.imshow("Result", c)
#cv2.waitKey(0)

In [7]:
#plt.imshow(c)